# Create TLE for the last known day a satellite was in orbit

### Traspose the Satellite Catalogue to a Dictionary


In [ ]:
import urllib.request
import pandas as pd

# Load the TSV file
tsv_file_path = './update_tle.tsv'

try:
    sheet_values = pd.read_csv(tsv_file_path, sep='\t')
except FileNotFoundError:
    sheet_values = pd.DataFrame(columns=['tle_0', 'tle_1', 'norad_cat_num'])

# Convert to dictionary for quick lookup
tle_dict = {row['norad_cat_num']: row for _, row in sheet_values.iterrows()}

# Fetch satellite catalog
satcat_url = "https://celestrak.com/pub/satcat.txt" # https://api.optimize.esa.int/data/debris_clouds/satcat.txt
try:
    sat_catalogue = urllib.request.urlopen(satcat_url).read().decode('utf-8').strip().split("\n")
except Exception as e:
    raise ConnectionError(f"Failed to fetch satellite catalog: {e}")

sat_dict = {}
for x in sat_catalogue:
    try:
        sat_dict[x.split()[1]] = x
    except:
        continue


### Login to Spacetrack


In [ ]:
import os
from dotenv import load_dotenv
from spacetrack import SpaceTrackClient

load_dotenv('.env') 

username = os.getenv('SPACETRACK_USERNAME')
password = os.getenv('SPACETRACK_PASSWORD')

st = SpaceTrackClient(username, password)

### Load Satellites to skip as they have been already processed

In [ ]:
import json

skipped_file = "skipped_satellites.json"

try:
    with open(skipped_file, 'r') as file:
        skipped_satellites = json.load(file)
except FileNotFoundError:
    skipped_satellites = []

### Save the TLE to an array


In [7]:
import time
from tqdm import tqdm

new_entries = []

for key in tqdm(sat_dict.keys(), desc="Processing satellites"):
    if str('n_' + key) not in tle_dict and key not in skipped_satellites:
        try:
            if "-" in sat_dict[key][75:85] and "N/A" not in sat_dict[key][119:127]:
                lines = st.tle_latest(norad_cat_id=[key], format='tle')
                lines = lines.split("\n")
                
                for line1, line2 in zip(*[iter(lines[-3:-1])]*2):
                    update_line = [str(line1), str(line2), str('n_' + key)]
                    # print(update_line)
                    new_entries.append(update_line)
                    skipped_satellites.append(key)
                    # Limit the API request
                    time.sleep(2) # 20 advised
            else:
                skipped_satellites.append(key)
                continue
        except Exception as e:
            print(f"Error processing satellite {key}: {e}")
            skipped_satellites.append(key)
    else:
         skipped_satellites.append(key)

Processing satellites:  14%|█▍        | 8312/59865 [00:31<00:40, 1286.50it/s]

### Save the satellites to skip into a json


In [5]:
with open(skipped_file, 'w') as file:
    json.dump(skipped_satellites, file)

### Save a csv for the TLEs

In [6]:

if new_entries:
    new_df = pd.DataFrame(new_entries, columns=['tle_0', 'tle_1', 'norad_cat_num'])
    updated_sheet = pd.concat([sheet_values, new_df])
    updated_sheet.to_csv(tsv_file_path, sep='\t', index=False)
else:
    print("No new entries to update.")